# Required Packages
* Imbalanced data handling: *imbalanced-learn*
* Classification model: *xgboost*
* Model explanation: *shap*

In [ ]:
%pip install --upgrade pip imblearn xgboost shap Geohash ray optuna

# Define Utility Functions

In [ ]:
import pytz
import pickle5 as pickle
import cloudpickle
import ray
import warnings
from datetime import datetime
import mitosheet
import pandas as pd
import numpy as np
import scipy.stats as st
from functools import wraps
from contextlib import contextmanager


DEFAULT_TZ = pytz.FixedOffset(540)  # GMT+09:00; Asia/Seoul


def load(path: str):
    with open(path, mode='rb') as f:
        return pickle.load(f)

    
def dump(obj, path: str):
    with open(path, mode='wb') as f:
        cloudpickle.dump(obj, f)
        
    
def log(msg: any):
    print('[{}] {}'.format(datetime.now().strftime('%y-%m-%d %H:%M:%S'), msg))

    
def show(d: pd.DataFrame):
    return mitosheet.sheet(d)


def stat(x):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        m = np.mean(x)
        s = np.std(x, ddof=1)
        l, u = st.t.interval(0.95, len(x) - 1, loc=np.mean(x), scale=st.sem(x))
        med = np.median(x)
        q_1, q_3 = np.quantile(x, [0.25, 0.75])
        iqr = (q_3 - q_1) * 1.5
    
        return m, s, l, u, med, q_1 - iqr, q_3 + iqr


@contextmanager
def on_ray(*args, **kwargs):
    try:
        if ray.is_initialized():
            ray.shutdown()
        ray.init(*args, **kwargs)
        yield None
    finally:
        ray.shutdown()
        
        
def catch_warning(t: str):
    def wrapper(f):
        @wraps(f)
        def decorator(*args, **kwargs):
            with warnings.catch_warnings():
                warnings.simplefilter(t)
                return f(*args, **kwargs)
        return decorator
    return wrapper

# Dataset Overview

## Raw records as csv format

### Label data
* EsmResponse-1966

### Demographic information
* UserInfo

### Smartphone data
* AppUsageEvent
* AppUsageStat
* BatteryEvent
* CallEvent
* Connectivity
* DataTraffic
* DeviceEvent
* InstalledApp
* Location
* MediaEvent
* MessageEvent
* ActivityEvent
* ActivityTransition
* WiFi


### MicrosoftBand2 data
* Acceleration
* AmbientLight
* Calorie
* Distance
* EDA
* HR
* StepCount
* RRI
* SkinTemperature
* UltraViolet

## Labels

The affect state labels (e.g., valence, arousal, etc.) and the corresponding collection times are retrieved from the ESM response data. 

In [ ]:
import pandas as pd
import numpy as np


l = pd.read_csv('./data/EsmResponse-1966.csv')
LABEL = pd.merge(
    left=l, 
    right=l.groupby(
        'Pcode', as_index=False
    ).mean().rename(lambda x: f'_{x}', axis=1),
    left_on='Pcode', 
    right_on='_Pcode'
).assign(
    pcode=lambda x: x['Pcode'],
    timestamp=lambda x: pd.to_datetime(x['ResponseTime'], unit='ms', utc=True).dt.tz_convert(DEFAULT_TZ),
    val_dyn=lambda x: np.where(x['Valence'] > x['_Valence'], 1, 0),
    aro_dyn=lambda x: np.where(x['Arousal'] > x['_Arousal'], 1, 0),
    att_dyn=lambda x: np.where(x['Attention'] > x['_Attention'], 1, 0),
    sts_dyn=lambda x: np.where(x['Stress'] > x['_Stress'], 1, 0),
    dst_dyn=lambda x: np.where(x['Disturbance'] > x['_Disturbance'], 1, 0),
    val_fix=lambda x: np.where(x['Valence'] > 0, 1, 0),
    aro_fix=lambda x: np.where(x['Arousal'] > 0, 1, 0),
    att_fix=lambda x: np.where(x['Attention'] > 0, 1, 0),
    sts_fix=lambda x: np.where(x['Stress'] > 0, 1, 0),
    dst_fix=lambda x: np.where(x['Disturbance'] > 0, 1, 0),
).set_index(
    ['pcode', 'timestamp']
)[[
    'val_dyn', 'aro_dyn', 'att_dyn', 'sts_dyn', 'dst_dyn',
    'val_fix', 'aro_fix', 'att_fix', 'sts_fix', 'dst_fix'
]]


dump(LABEL, './proc/label.pkl')

In [ ]:
LABEL

In [ ]:
import seaborn as sns

esm = pd.read_csv("./data/EsmResponse-1966.csv")

box = sns.boxplot(data=esm[["Valence", "Arousal", "Stress", "Attention", "Disturbance", "Change"]],
                  showmeans=True,
                  meanprops={"marker":"o",
                             "markerfacecolor":"white", 
                             "markeredgecolor":"black",
                             "markersize":"7"}
                 )
# box.title.set_text(f"Label Distribution (1)")

In [ ]:
import seaborn as sns

esm = pd.read_csv("./data/EsmResponse-1966.csv")

box = sns.boxplot(data=esm[["Duration"]],
                  showmeans=True,
                  meanprops={"marker":"o",
                             "markerfacecolor":"white", 
                             "markeredgecolor":"black",
                             "markersize":"7"},
                  width=.13,
                  color='grey'
                 )
# box.title.set_text(f"Label Distribution (2)")

## Feature candidates

### Participants' information

In [ ]:
PARTICIPANT_INFO = pd.read_csv("./data/UserInfo.csv")

In [ ]:
PARTICIPANT_INFO

### Sensor data retrieval

In [ ]:
import os
import pandas as pd
import numpy as np
from typing import Dict, Optional, Union, Iterable
import Geohash as geo
import json
from datetime import timedelta
    

def _load_data(
    name: str,
    conds: Dict[str, Union[Iterable[str], str]] = None
) -> Optional[pd.DataFrame]:
    paths = [
        (d, os.path.join('data', d, f'{name}.csv'))
        for d in os.listdir('data')
        if d.startswith('P')
    ]
    data = pd.concat([
        pd.read_csv(p).assign(pcode=pcode)
        for pcode, p in paths
        if os.path.exists(p)
    ], ignore_index=True)
    
    if conds is not None:
        for k, v in conds.items():
            if type(v) is str:
                data = data.loc[lambda x: x[k] == v, :]
            else:
                data = data.loc[lambda x: x[k].isin(v), :]
                
    data = data.assign(
        timestamp=lambda x: pd.to_datetime(x['timestamp'], unit='ms', utc=True).dt.tz_convert(DEFAULT_TZ)
    ).set_index(
        ['pcode', 'timestamp']
    )
    
    return data


def _load_app_usage() -> Dict[str, pd.Series]:
    data = _load_data(name='AppUsageEvent', conds={
        'type': ['MOVE_TO_FOREGROUND', 'MOVE_TO_BACKGROUND']
    }).assign(
        raw=lambda x: np.where(x['type'] == 'MOVE_TO_FOREGROUND', x['packageName'], None),
        #cls=lambda x: np.where(x['type'] == 'MOVE_TO_FOREGROUND', x['packageName'].replace(category), None),
    )

    return {
        'PHO#APP_RAW': data['raw'].astype('object'),
        'PHO#APP_CLS': data['category'].astype('object')
    }


def _load_connectivity() -> Dict[str, pd.Series]:
    data = _load_data(name='Connectivity').assign(
        type=lambda x: np.where(x['isConnected'] == True, x['type'], 'DISCONNECTED')
    ).replace({
        'type': {
            'MOBILE': 'MOBILE',
            'MOBILE_DUN': 'MOBILE',
            'UNDEFINED': 'DISCONNECTED',
            'VPN': 'MOBILE',
            'WIFI': 'WIFI'
        }
    })

    return {
        'PHO#CON': data['type'].astype('object')
    }


def _load_battery() -> Dict[str, pd.Series]:
    data = _load_data(name='BatteryEvent').assign(
        temperature=lambda x: (x['temperature'] / 10.0).astype(np.float64)
    )

    return {
        'PHO#BAT_LEV': data['level'].astype('float64'),
        'PHO#BAT_CHG': data['status'].astype('object'),
        'PHO#BAT_TMP': data['temperature'].astype('float64')
    }
        

def _load_call() -> Dict[str, pd.Series]:
    data = _load_data(name='CallEvent', conds={
        'type': ['OUTGOING', 'INCOMING']
    }).reset_index()
        
    new_data = []
        
    for pcode in data['pcode'].unique():
        sub = data.loc[
            lambda x: x['pcode'] == pcode, :
        ].sort_values(
            'timestamp'
        )
        for row in sub.itertuples():
            new_data.append({
                'pcode': row.pcode,
                'timestamp': row.timestamp,                
                'type': row.type,
            })
            new_data.append({
                'pcode': row.pcode,
                'timestamp': row.timestamp + timedelta(seconds=row.duration),
                'type': 'IDLE'
            })
    new_data = pd.DataFrame(new_data).set_index(
        ['pcode', 'timestamp']
    )

    return {
        'PHO#CAL': new_data['type'].astype('object'),
    }


def _load_data_traffic() -> Dict[str, pd.Series]:
    data = _load_data(name='DataTraffic')

    return {
        'PHO#DAT_RCV': data['rxKiloBytes'].astype('float64'),
        'PHO#DAT_SNT': data['txKiloBytes'].astype('float64')
    }


def _load_device_event() -> Dict[str, pd.Series]:
    data = _load_data(name='DeviceEvent')
    
    scr = data.loc[
        lambda x: x['type'].isin(['SCREEN_ON', 'SCREEN_OFF', 'UNLOCK']), :
    ].assign(
        type=lambda x: x['type'].str.replace('SCREEN_', '')
    )
    
    rng = data.loc[
        lambda x: x['type'].isin(['RINGER_MODE_VIBRATE', 'RINGER_MODE_SILENT', 'RINGER_MODE_NORMAL']), :
    ].assign(
        type=lambda x: x['type'].str.replace('RINGER_MODE_', '')
    )
    
    return {
        'PHO#SCR': scr['type'].astype('object'),
        'PHO#RNG': rng['type'].astype('object'),
    }


def _load_location() -> Dict[str, pd.Series]:
    def _haversine(_lat1: float, _lat2: float, _lng1: float, _lng2: float) -> float:
        if np.isnan(_lat1) or np.isnan(_lat2) or np.isnan(_lng1) or np.isnan(_lng2):
            return 0.0

        _lat1_r, _lat2_r, _lng1_r, _lng2_r = np.radians(_lat1), np.radians(_lat2), np.radians(_lng1), np.radians(_lng2)
        _lat = _lat2_r - _lat1_r
        _lng = _lng2_r - _lng1_r
        _R = 6371008.8
        _d = np.sin(_lat * 0.5) ** 2 + np.cos(_lat1_r) * np.cos(_lat2_r) * np.sin(_lng * 0.5) ** 2
        return 2 * _R * np.arcsin(np.sqrt(_d))

    data = _load_data(name='Location').reset_index()
    new_data = []

    for pcode in data['pcode'].unique():
        sub = data.loc[
            lambda x: x['pcode'] == pcode, :
        ].sort_values(
            'timestamp'
        )

        new_sub = pd.concat([
            sub, sub.rename(lambda x: f'_{x}', axis=1).shift(-1)
        ], axis=1).assign(
            dist=lambda x: [
                _haversine(*_coord) for _coord
                in zip(x['latitude'], x['_latitude'], x['longitude'], x['_longitude'])
            ],
            cluster=lambda x: [
                geo.encode(lat, lon, precision=6)
                for lat, lon in zip(x['latitude'], x['longitude'])
            ]
        )

        for row in new_sub.itertuples():
            new_data.append({
                'pcode': row.pcode,
                'timestamp': row.timestamp,                
                'dist': row.dist,
                'cluster': row.cluster,
            })

    new_data = pd.DataFrame(new_data).set_index(
        ['pcode', 'timestamp']
    )

    return {
        'PHO#LOC_CLS': new_data['cluster'].astype('object'),
        'PHO#LOC_DST': new_data['dist'].astype('float64'),
    }


def _load_activity() -> Dict[str, pd.Series]:
    data = _load_data(name='ActivityTransition', conds={
        'transitionType': ['ENTER_WALKING', 'ENTER_STILL', 'ENTER_IN_VEHICLE', 'ENTER_ON_BICYCLE', 'ENTER_RUNNING']
    }).assign(
        type=lambda x: x['transitionType'].str.replace('ENTER_', '')
    )

    return {
        'PHO#ACT': data['type'].astype('object')
    }


def _load_accelerometer() -> Dict[str, pd.Series]:
    data = _load_data(name='Acceleration')
    return {
        'WEA#ACC_X': data['X'].astype('float64'),
        'WEA#ACC_Y': data['Y'].astype('float64'),
        'WEA#ACC_Z': data['Z'].astype('float64')        
    }


def _load_ultra_violet() -> Dict[str, pd.Series]:
    data = _load_data(name='UltraViolet')
    return {
        'WEA#ULB_IDX': data['UVIndexLevel'].astype('object'),
    }


def _load_skin_temperature() -> Dict[str, pd.Series]:
    data = _load_data(name='SkinTemperature')
    return {
        'WEA#TMP': data['Temperature'].astype('float64'),
    }


def _load_rr_interval() -> Dict[str, pd.Series]:
    data = _load_data(name='RRI')
    return {
        'WEA#RRI': data['Interval'].astype('float64'),
    }


def _load_brightness() -> Dict[str, pd.Series]:
    data = _load_data(name='AmbientLight')
    return {
        'WEA#BRI': data['Brightness'].astype('float64'),
    }


def _load_step_counts() -> Dict[str, pd.Series]:
    data = _load_data(name='StepCount').reset_index()
    new_data = []

    for pcode in data['pcode'].unique():
        sub = data.loc[
            lambda x: x['pcode'] == pcode, :
        ].sort_values(
            'timestamp'
        )

        new_sub = pd.concat([
            sub, sub.rename(lambda x: f'_{x}', axis=1).shift(-1)
        ], axis=1).assign(
            diff=lambda x: x['_TotalSteps'] - x['TotalSteps']
        )

        for row in new_sub.itertuples():
            new_data.append({
                'pcode': row.pcode,
                'timestamp': row.timestamp,                
                'diff': row.diff
            })

    new_data = pd.DataFrame(new_data).set_index(
        ['pcode', 'timestamp']
    )

    return {
        'WEA#STP': new_data['diff'].astype('float64'),
    }


def _load_heart_rate() -> Dict[str, pd.Series]:
    data = _load_data(name='HR')
    return {
        'WEA#HRT': data['BPM'].astype('float64')
    }


def _load_gsr() -> Dict[str, pd.Series]:
    data = _load_data(name='EDA')
    return {
        'WEA#GSR': data['Resistance'].astype('float64')
    }


def _load_distance() -> Dict[str, pd.Series]:
    data = _load_data(name='Distance').reset_index()
    new_data = []

    for pcode in data['pcode'].unique():
        sub = data.loc[
            lambda x: x['pcode'] == pcode, :
        ].sort_values(
            'timestamp'
        )

        new_sub = pd.concat([
            sub, sub.rename(lambda x: f'_{x}', axis=1).shift(-1)
        ], axis=1).assign(
            diff=lambda x: x['_TotalDistance'] - x['TotalDistance']
        )

        for row in new_sub.itertuples():
            new_data.append({
                'pcode': row.pcode,
                'timestamp': row.timestamp,                
                'diff': row.diff,
                'motion': row.MotionType,
                'pace': row.Pace,
                'speed': row.Speed,
            })

    new_data = pd.DataFrame(new_data).set_index(
        ['pcode', 'timestamp']
    )

    return {
        'WEA#ACT_DST': new_data['diff'].astype('float64'),
        'WEA#ACT_MOT': new_data['motion'].astype('object'),
        'WEA#ACT_PAC': new_data['pace'].astype('float64'),
        'WEA#ACT_SPD': new_data['speed'].astype('float64'),
    }


def _load_calories() -> Dict[str, pd.Series]:
    data = _load_data(name='Calorie').reset_index()
    new_data = []

    for pcode in data['pcode'].unique():
        sub = data.loc[
            lambda x: x['pcode'] == pcode, :
        ].sort_values(
            'timestamp'
        )

        new_sub = pd.concat([
            sub, sub.rename(lambda x: f'_{x}', axis=1).shift(-1)
        ], axis=1).assign(
            diff=lambda x: x['_TotalCalories'] - x['TotalCalories']
        )

        for row in new_sub.itertuples():
            new_data.append({
                'pcode': row.pcode,
                'timestamp': row.timestamp,                
                'diff': row.diff
            })

    new_data = pd.DataFrame(new_data).set_index(
        ['pcode', 'timestamp']
    )

    return {
        'WEA#CAL': new_data['diff'].astype('float64')       
    }


In [ ]:
import traceback as tb
from functools import reduce
from collections import defaultdict


DATA = defaultdict(dict)

func_loads = [
    _load_app_usage,
    _load_connectivity,
    _load_battery,
    _load_call,
    _load_data_traffic,
    _load_device_event,
    _load_location,
    _load_activity,
    _load_accelerometer,
    _load_ultra_violet,
    _load_skin_temperature,
    _load_rr_interval,
    _load_brightness,
    _load_step_counts,
    _load_heart_rate,
    _load_gsr,
    _load_distance,
    _load_calories
]

data = []

for func in func_loads:
    try:
        d = func()
    except:
        d = dict()
        log(f'Error occurs in data = {func.__name__}. Caused by: {tb.format_exc()}')
    finally:
        data.append(d)

data = reduce(lambda a, b: {**a, **b}, data)
for k, v in data.items():
    DATA[k] = v.sort_index()

for pcode in LABEL.index.droplevel('timestamp').unique():
    data_subject = dict()
    
    for k, v in DATA.items():
        try:
            data_subject[k] = v[pcode]
        except:
            pass
    
    dump(data_subject, f'./proc/raw_data-{pcode}.pkl')

del DATA

## Data distribution

### Sensor data from Smartphone and MS Band2

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta as td

LABEL = load('./proc/label.pkl')

EXP_DATE = LABEL.reset_index().groupby(
    ['pcode']
).agg(
    start_day=pd.NamedAgg(column='timestamp', aggfunc=lambda x: x.min().replace(hour=0, minute=0, second=0, microsecond=0)),
    end_day=pd.NamedAgg(column='timestamp', aggfunc=lambda x: (x.max() + td(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)),
)

DIST_DATA = []

for p in EXP_DATE.index:
    raw_data = load(f'./proc/raw_data-{p}.pkl')

    dist_temp = []

    for data_type, data_values in raw_data.items():
        data_sub = pd.concat([
            data_values.to_frame().reset_index().assign(pcode=p)
        ]).merge(
            EXP_DATE, on='pcode', how='left'
        ).loc[
            lambda x: ((x['timestamp'] - x['start_day']).dt.total_seconds() >= 0) & ((x['end_day'] - x['timestamp']).dt.total_seconds() >= 0), :
        ].assign(
            day = lambda x: np.floor((x['timestamp'] - x['start_day']).dt.total_seconds() / 60 / 60 / 24)
        ).groupby(
            ['pcode', 'day']
        ).count()['timestamp'].reset_index().assign(
            type=data_type
        )

        dist_temp.append(data_sub)

    DIST_DATA.append(pd.concat(dist_temp))
    
DIST_DATA = pd.concat(DIST_DATA)        

In [ ]:
DIST_DATA

In [ ]:
DIST_DATA.groupby(
        ['type', 'pcode']
    ).sum()['timestamp'].reset_index().groupby(
        'type'
    )['timestamp'].agg(
        lambda x: 'N={0:.2f}; mean={1[0]:.2f} (SD: {1[1]:.2f}; [{1[2]:.2f}, {1[3]:.2f}]); med={1[4]:.2f} ([{1[5]:.2f}, {1[6]:.2f}])'.format(
            np.sum(x), stat(x)
        )
    ).reset_index()

In [ ]:
import altair as alt

alt.data_transformers.disable_max_rows()

selection = alt.selection_single(
    fields=['type'],
    bind=alt.binding_select(
        options=DIST_DATA['type'].unique(),
        name='Data Type: '
    )
)

alt.Chart(DIST_DATA).mark_rect().encode(
     x=alt.X('day:O', title='Time (days)'),
     y=alt.Y('pcode:O', title='Pcode'),
     color=alt.Color('timestamp:Q', title='Count')
).transform_filter(
    selection
).add_selection(
    selection
).properties(
    title='# Instances',
    width=300,
    height=800
)

As shown above, PHO#CAL (Call log) and PHO#RNG (Ringer mode) were not regularly collected (i.e., too many missing values).

In [ ]:
from collections import defaultdict


EXCLUDE_DTYPE = ['PHO#CAL', 'PHO#RNG']
LABEL = load('./proc/label.pkl')

for pcode in LABEL.index.droplevel('timestamp').unique():
    data = load(f'./proc/raw_data-{pcode}.pkl')
    data = {
        k: v
        for k, v in data.items()
        if k not in EXCLUDE_DTYPE
    }
    dump(data, f'./proc/clean_data-{pcode}.pkl')

# Feature Extraction

## Implementation

In [ ]:
import numpy as np
import pandas as pd
from typing import Dict, Callable, Union, Tuple, List, Optional
from datetime import timedelta as td
from scipy import stats
import time


def extract(
        pid: str,
        data: Dict[str, pd.Series],
        label: pd.Series,        
        window_data: Dict[str, Union[int, Callable[[pd.Timestamp], int]]],
        window_label: Dict[str, Union[int, Callable[[pd.Timestamp], int]]],        
        categories: Dict[str, Optional[List[any]]] = None,
        pinfo: Dict[str, any] = None,
        resample_s: int = -1,
        verbosity: int = -1
) -> Tuple[pd.DataFrame, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    def _log(_msg, _verb):
        _prefix = '[DEBUG]' if _verb == 0 else '[INFO]'
        if _verb <= verbosity:
            log(f'{_prefix} {_msg}')

    elapsed_time = time.time()
    _log(f'Start feature extraction for pid == {pid}', 0)
    
    categories = categories or dict()
    pinfo = pinfo or dict()
    data_resampled = dict()
    
    for k, v in data.items():
        v = v.loc[lambda x: ~x.index.duplicated(keep=False)]
        if k in categories:
            data_resampled[k] = v.resample(f'{resample_s}S').ffill().dropna() if resample_s > 0 and len(v) > 0 else v
        else:
            data_resampled[k] = v.resample(f'{resample_s}S').mean().interpolate(method='linear').dropna() if resample_s > 0 and len(v) > 0 else v
    _log(f'Data resampled', 1)

    X, y, group, date_times = [], [], [], []

    for t_label, v_label in zip(label.index, label.values):
        row = dict()

        # Features relevant to participants' info
        for d_prefix, d_value in pinfo.items():
            row[f'PIF#{d_prefix}'] = d_value
        _log(f'Extracted feature relevant to personal info at {t_label}', 1)

        # Features relevant to time
        day_of_week = ['MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN'][t_label.isoweekday() - 1]
        is_weekend = 'Y' if t_label.isoweekday() > 5 else 'N'
        hour = t_label.hour

        if 6 <= hour < 9:
            hour_name = 'DAWN'
        elif 9 <= hour < 12:
            hour_name = 'MORNING'
        elif 12 <= hour < 15:
            hour_name = 'AFTERNOON'
        elif 15 <= hour < 18:
            hour_name = 'LATE_AFTERNOON'
        elif 18 <= hour < 21:
            hour_name = 'EVENING'
        elif 21 <= hour < 24:
            hour_name = 'NIGHT'
        else:
            hour_name = 'MIDNIGHT'
        
        row['TIM#DOW'] = day_of_week
        row['TIM#WKD'] = is_weekend
        row['TIM#HRN'] = hour_name
        _log(f'Extracted feature relevant to delivery time at {t_label}', 1)

        # Features relevant to latest value of a given smartphone data
        for d_prefix, d_value in data_resampled.items():
            if d_prefix in categories and categories[d_prefix] is None:
                continue
                
            d = d_value.sort_index().loc[:t_label]
            v = None
            if len(d):
                v = d.iloc[-1]
            else:
                if d.dtype == object:
                    v = 'UNKNOWN'
                else:
                    v = 0.0
            row[f'{d_prefix}#VAL'] = v
        _log(f'Extracted feature relevant to latest smartphone value at {t_label}', 1)

        # Features relevant to duration since the latest state change (only for categorical smartphone data)
        for d_prefix, d_value in data_resampled.items():
            if d_prefix not in categories:
                continue

            d = d_value.sort_index().loc[:t_label]
            row[f'{d_prefix}#DLC'] = (t_label - d.loc[lambda x: x == x.iloc[-1]].index.min()).total_seconds() if len(d) else -1

            cats = categories[d_prefix] or list()
            for c in cats:
                d = d_value.sort_index().loc[:t_label].loc[lambda x: x == c]
                row[f'{d_prefix}#DLC={c}'] = (t_label - d.index[-1]).total_seconds() if len(d) else -1

            _log(f'Extracted feature relevant to duration since the latest change at {t_label} for {d_prefix}', 1)

        # Features extracted from windowed smartphone data
        for w_suffix, w_value in window_data.items():
            w_value = w_value(t_label) if isinstance(w_value, Callable) else w_value

            for d_prefix, d_value in data_resampled.items():
                d = d_value.loc[(t_label - td(seconds=w_value)):t_label]

                if len(d) == 0:
                    continue
                
                if d_prefix in categories:
                    cnt = d.value_counts()
                    val, sup = cnt.index, cnt.values
                    hist = {k: v for k, v in zip(val, sup)}

                    # Information Entropy
                    row[f'{d_prefix}#ETP#{w_suffix}'] = stats.entropy(sup / len(d)) 
                    
                    # Abs. Sum of Changes
                    row[f'{d_prefix}#ASC#{w_suffix}'] = np.sum(d.values[1:] != d.values[:-1]) 

                    cats = categories[d_prefix] or list()
                    
                    # Duration
                    for c in cats:
                        row[f'{d_prefix}#DUR={c}#{w_suffix}'] = hist[c] if c in hist else 0

                else:
                    d = d.astype('float32').values
                    hist, _ = np.histogram(d, bins='doane', density=False)
                    std = np.sqrt(np.var(d, ddof=1)) if len(d) > 1 else 0
                    norm_x = (d - np.mean(d)) / std if std != 0 else np.zeros(len(d))

                    row[f'{d_prefix}#AVG#{w_suffix}'] = np.mean(d) # Sample mean
                    row[f'{d_prefix}#STD#{w_suffix}'] = std # Sample standard deviation
                    row[f'{d_prefix}#SKW#{w_suffix}'] = stats.skew(d, bias=False) # Sample skewness
                    row[f'{d_prefix}#KUR#{w_suffix}'] = stats.kurtosis(d, bias=False) # Sample kurtosis
                    row[f'{d_prefix}#ASC#{w_suffix}'] = np.sum(np.abs(np.diff(d))) # Abstract sum of changes
                    row[f'{d_prefix}#BEP#{w_suffix}'] = stats.entropy(hist) # Binned entropy
                    row[f'{d_prefix}#MED#{w_suffix}'] = np.median(d) # Median
                    row[f'{d_prefix}#TSC#{w_suffix}'] = np.sqrt(np.sum(np.power(np.diff(norm_x), 2))) # Timeseries complexity
                
                _log(f'Extracted feature relevant to a time-window data at {t_label} for {d_prefix} and {w_suffix}', 1)
    
        # Features extracted from previous respones behavior toward JIT intervention
        for w_suffix, w_value in window_label.items():
            w_value = w_value(t_label) if isinstance(w_value, Callable) else w_value
            d = label.loc[(t_label - td(seconds=w_value)):t_label]
            row[f'REC#LIK#{w_suffix}'] = np.sum(d == 1.0) / len(d) if len(d) > 0 else 0
            _log(f'Extracted feature relevant to a time-window resp. behavior at {t_label} for {w_suffix}', 1)

        X.append(row)
        y.append(v_label)
        group.append(pid)
        date_times.append(t_label)

    X, y, group, date_times = pd.DataFrame(X), np.asarray(y), np.asarray(group), np.asarray(date_times)
    t_s = date_times.min().replace(hour=0, minute=0, second=0, microsecond=0).timestamp()
    timestamps = np.asarray(list(map(lambda x: x.timestamp() - t_s, date_times)))
   
    dtypes = {
        k: 'object' if v == object else 'float32'
        for k, v in X.dtypes.items()
    }
    C = X.columns
    C_cat = list(sorted([c for c in C if dtypes[c] == object]))
    C_num = list(sorted([c for c in C if dtypes[c] != object]))
    X_C, X_N = X[C_cat], X[C_num]
    X_C = X_C.fillna(value='UNKNOWN')
    X_N = X_N.fillna(value=0.0)
    X = pd.DataFrame(np.concatenate([X_C, X_N], axis=1), columns=C_cat + C_num).astype(dtypes)
    
    _log(f'Completed feature extraction for pid == {pid} ({time.time() - elapsed_time} s)', 0)
    return X, y, group, timestamps, date_times


## Execution with *ray* module

In [ ]:
import pandas as pd
import numpy as np
import ray
from collections import defaultdict


SUBJECTS = pd.read_csv('./data/UserInfo.csv')[[
    'Pcode', 'Age', 'Gender', 'Openness', 'Conscientiousness', 
    'Neuroticism', 'Extraversion', 'Agreeableness', 'PSS10', 'PHQ9', 'GHQ12'
]].rename(columns={
    'Age': 'AGE', 'Gender': 'GEN', 
    'Openness': 'BFI_OPN', 'Conscientiousness': 'BFI_CON', 
    'Neuroticism': 'BFI_NEU', 'Extraversion': 'BFI_EXT',
    'Agreeableness': 'BFI_AGR', 
    'PSS10': 'PSS', 'PHQ9': 'PHQ', 'GHQ12': 'GHQ'
}).set_index(
    'Pcode'
).to_dict('index')

LABEL = load('./proc/label.pkl')

PCODES = LABEL.index.droplevel('timestamp').unique()

WIN_DATA = {
    'M1': 60,
    'M5': 60 * 5,
    'M10': 60 * 10,
    'M30': 60 * 30,
    'H1': 60 * 60,
    'H3': 60 * 60 * 3,
    'H6': 60 * 60 * 6,
    'D1': 60 * 60 * 24
}

WIN_LABEL = {
    'H3': 60 * 60 * 3,
    'H6': 60 * 60 * 6,
    'H12': 60 * 60 * 12,
    'D1': 60 * 60 * 24,
    'D3': 60 * 60 * 24 * 3
}


CATEGORIES = defaultdict(list)
CATEGORIES['PHO#APP_RAW'] = None
CATEGORIES['PHO#LOC_CLS'] = None

for pcode in PCODES:
    d = load(f'./proc/clean_data-{pcode}.pkl')
    
    for c in [
        'PHO#APP_CLS', 'PHO#CON', 'PHO#BAT_CHG', 'PHO#SCR', 'PHO#ACT', 'WEA#ULB_IDX', 'WEA#ACT_MOT'
    ]:
        CATEGORIES[c].extend(list(filter(lambda x: x is not None, d[c].unique())))

CATEGORIES = {
    k: None if v is None else list(set(v))
    for k, v in CATEGORIES.items()
}

In [ ]:
with on_ray(ignore_reinit_error=True, num_cpus=7):
    func = ray.remote(extract).remote

    for l in LABEL.columns:
        jobs = []
        
        for pcode in PCODES:
            pinfo = SUBJECTS[pcode]
            data = load(f'./proc/clean_data-{pcode}.pkl')
            label = LABEL[l].loc[pcode]
            
            job = func(
                pid=pcode,
                data=data,
                label=label,
                window_data=WIN_DATA,
                window_label=WIN_LABEL,
                categories=CATEGORIES,
                pinfo=pinfo,
                resample_s=1,
                verbosity=0
            )
            jobs.append(job)
            
        jobs = ray.get(jobs)

        X = pd.concat(list(map(lambda x: x[0], jobs)), ignore_index=True)
        y = np.concatenate(list(map(lambda x: x[1], jobs)), axis=0)
        group = np.concatenate(list(map(lambda x: x[2], jobs)), axis=0)
        t = np.concatenate(list(map(lambda x: x[3], jobs)), axis=0)
        date_times = np.concatenate(list(map(lambda x: x[4], jobs)), axis=0)
        
        dataset = dict(
            X=X,
            y=y,
            group=group,
            t=t,
            date_times=date_times
        )
        dump(dataset, f'./proc/dataset.{l}.pkl')

# Feature Exploration

In [ ]:
import pandas as pd
import numpy as np
import altair as alt
from typing import Iterable


def vis_feature(
    X: pd.DataFrame,
    y: np.ndarray,
    pids: np.ndarray,    
    feature: str
): 
    data = X.assign(
        label = y,
        pids = pids
    )[[feature, 'label', 'pids']]
        
    dtype = data[feature].dtype
    
    if dtype == object:
        data_all = data.assign(
            cnt=0
        ).groupby(
            ['label', feature], as_index=False
        ).count()
        
        data_sub = data.assign(
            cnt=0
        ).groupby(
            ['label', 'pids', feature], as_index=False
        ).count()
        
        c_all = alt.Chart(
            data_all
        ).encode(
            x=alt.X('label:N', title='Class'),
            y=alt.Y('cnt:Q', title='Ratio', stack='normalize'),
            color=f'{feature}:N'
        ).mark_bar().properties(
            width=30,
            height=200,
        )
        
        c_sub = alt.Chart(
            data_sub
        ).encode(
            x=alt.X('label:N', title='Class'),
            y=alt.Y('cnt:Q', title='Ratio', stack='normalize'),
            color=f'{feature}:N'                    
        ).mark_bar().properties(
            width=30,
            height=200
        ).facet(
            column=alt.Column('pids:N', title='PIDs')
        )
        
        return c_all | c_sub
        
    else:
        c_all = alt.Chart(
            data
        ).encode(
            x=alt.X('label:N', title='Class'),
            y=alt.Y(f'{feature}:Q', title='Value'),
            color=alt.Color('label:N')
        ).mark_boxplot(size=10).properties(
            width=30,
            height=200
        )
        
        c_sub = alt.Chart(
            data
        ).encode(
            x=alt.X('label:N', title='Class'),
            y=alt.Y(f'{feature}:Q', title='Value'),
            color=alt.Color('label:N', title='Class')
        ).mark_boxplot(size=10).properties(
            width=30,
            height=200,
        ).facet(
            column=alt.Column('pids:N', title='PIDs')
        )
        
        return c_all | c_sub


## Arousal

In [ ]:
from ipywidgets import interact, fixed, widgets


DATA_ARO_FIX = load('./proc/dataset.aro_fix.pkl')

interact(
    vis_feature,
    X=fixed(DATA_ARO_FIX['X']),
    y=fixed(DATA_ARO_FIX['y']),
    pids=fixed(DATA_ARO_FIX['group']),
    feature=widgets.Select(options=sorted(DATA_ARO_FIX['X'].columns), rows=15)
)

## Valence

In [ ]:
from ipywidgets import interact, fixed, widgets


DATA_VAL_DYN = load('./proc/dataset.val_fix.pkl')

interact(
    vis_feature,
    X=fixed(DATA_VAL_DYN['X']),
    y=fixed(DATA_VAL_DYN['y']),
    pids=fixed(DATA_VAL_DYN['group']),
    feature=widgets.Select(options=sorted(DATA_VAL_DYN['X'].columns), rows=15)
)

## Stress

In [ ]:
from ipywidgets import interact, fixed, widgets


DATA_STS_FIX = load('./proc/dataset.sts_fix.pkl')

interact(
    vis_feature,
    X=fixed(DATA_STS_FIX['X']),
    y=fixed(DATA_STS_FIX['y']),
    pids=fixed(DATA_STS_FIX['group']),
    feature=widgets.Select(options=sorted(DATA_STS_FIX['X'].columns), rows=15)
)

## Disturbance

In [ ]:
from ipywidgets import interact, fixed, widgets


DATA_DST_FIX = load('./proc/dataset.dst_fix.pkl')

interact(
    vis_feature,
    X=fixed(DATA_DST_FIX['X']),
    y=fixed(DATA_DST_FIX['y']),
    pids=fixed(DATA_DST_FIX['group']),
    feature=widgets.Select(options=sorted(DATA_DST_FIX['X'].columns), rows=15)
)

# Modeling Pipeline

## Retrieve and summarize the dataset

In [ ]:
import numpy as np


LABEL = load('./proc/label.pkl')

for l in LABEL.columns:
    dataset = load(f'./proc/dataset.{l}.pkl')
    X = dataset['X']
    space = X.shape
    cat_masks = X.dtypes == 'object'
    num_masks = X.dtypes != 'object'
    X_cat = X.loc[:, cat_masks]
    cardinality = []
    
    print(f'{"-" * 5}{l}{"-" * 5}')
    print(f'Feature space: {X.shape}')
    print(f'- Numeric: {np.sum(num_masks)}')
    print(f'- Categorical: {np.sum(cat_masks)}')
    
    for k, v in X_cat.iteritems():
        c = v.unique()
        print(f'-- {k}: {c}')
        cardinality.append(len(c))
    print(f'- Avg. cardinality: {np.mean(cardinality)}')
    
    k, c = np.unique(LABEL[l], return_counts=True)
    print(f'Class values: {k} / Dist. : {c}\r\n')

## Cross-validation

### Model definition

In [ ]:
import pandas as pd
import numpy as np
from typing import List, Iterable, Optional
from xgboost import XGBClassifier
from sklearn.base import BaseEstimator
from sklearn.model_selection import StratifiedShuffleSplit
from functools import partial


class ModifiedXGBClassifier(BaseEstimator):
    def __init__(
        self,
        eval_set=False,
        eval_metric='logloss',
        early_stopping_rounds=10,
        random_state=None,
        **kwargs
        ):
        self.random_state = random_state
        self.eval_set = eval_set
        self.eval_metric = eval_metric
        self.early_stopping_rounds = early_stopping_rounds
        self.model = XGBClassifier(
            random_state=self.random_state,
            eval_metric=self.eval_metric,
            early_stopping_rounds=self.early_stopping_rounds,
            **kwargs
        )

    @property
    def classes_(self):
        return self.model.classes_

    def fit(self, X: pd.DataFrame, y: np.ndarray):
        if self.eval_set:
            I_train, I_eval = next(StratifiedShuffleSplit(random_state=self.random_state).split(X, y))
            X_train, y_train = X.iloc[I_train, :], y[I_train]
            X_eval, y_eval = X.iloc[I_eval, :], y[I_eval]
            self.model = self.model.fit(
                X=X_train, y=y_train, eval_set=[(X_eval, y_eval)],
                verbose=False
            )
        else:
            self.model = self.model.fit(X=X, y=y)
        return self

    def predict(self, X: pd.DataFrame):
        return self.model.predict(X)

    def predict_proba(self, X: pd.DataFrame):
        return self.model.predict_proba(X)


### CV pipeline

In [ ]:
import os
import warnings
import traceback as tb
from functools import partial
from typing import Tuple, Dict, Union, Callable, Generator, List
from dataclasses import dataclass
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE, SMOTENC
from pyparsing import col
import ray
from ray import tune as ray_tune
from ray.tune.suggest.optuna import OptunaSearch
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold, LeaveOneGroupOut, StratifiedShuffleSplit, RepeatedStratifiedKFold, \
    StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.metrics import log_loss
from sklearn.base import BaseEstimator
import optuna
from optuna.samplers import TPESampler
import time


@dataclass
class FoldResult:
    name: str
    estimator: BaseEstimator
    X_train: pd.DataFrame
    y_train: np.ndarray
    X_test: pd.DataFrame
    y_test: np.ndarray
    tuned_params: Dict[str, any] = None
    categories: Dict[str, Dict[int, str]] = None

def _split(
        alg: str,
        X: Union[pd.DataFrame, np.ndarray] = None,
        y: np.ndarray = None,
        groups: np.ndarray = None,
        random_state: int = None,
        n_splits: int = None,
        n_repeats: int = None,
        test_ratio: float = None,
        t_size_s: int = None
) -> Generator[Tuple[np.ndarray, np.ndarray], None, None]:
    if alg == 'holdout':
        splitter = StratifiedShuffleSplit(
            n_splits=n_splits,
            test_size=test_ratio,
            random_state=random_state
        )
    elif alg == 'kfold':
        if n_repeats and n_repeats > 1:
            splitter = RepeatedStratifiedKFold(
                n_splits=n_splits,
                n_repeats=n_repeats,
                random_state=random_state,
            )
        else:
            splitter = StratifiedKFold(
                n_splits=n_splits,
                random_state=random_state,
                shuffle=False if random_state is None else True,
            )
    elif alg == 'logo':
        splitter = LeaveOneGroupOut()
    elif alg == 'groupk':
        splitter = StratifiedGroupKFold(
            n_splits=n_splits,
            random_state=random_state,
            shuffle=False if random_state is None else True
        )

    else:
        raise ValueError('"alg" should be one of "holdout", "kfold", "logo", or "groupk".')

    split = splitter.split(X, y, groups)

    for I_train, I_test in split:
        yield I_train, I_test


def _trainable(
    config,
    checkpoint_dir=None,
    X: pd.DataFrame = None,
    y: np.ndarray = None,
    categories: List[str] = None,
    split: Callable[[pd.DataFrame, np.ndarray], Generator[Tuple[np.ndarray, np.ndarray], None, None]] = None,
    fit: Callable[[pd.DataFrame, np.ndarray, Dict[str, any], List[str]], BaseEstimator] = None,
    ignore_warning: bool = False,
):
    losses = []

    with warnings.catch_warnings():
        if ignore_warning:
            warnings.simplefilter('ignore')

        for I_train, I_eval in split(X, y):
            X_train, y_train, X_eval, y_eval = X.iloc[I_train, :], y[I_train], X.iloc[I_eval, :], y[I_eval]
            estimator = fit(X_train, y_train, config, categories)

            if estimator is None:
                break

            y_pred = estimator.predict_proba(X_eval)
            losses.append(log_loss(y_true=y_eval, y_pred=y_pred))

    losses = np.nanmean(losses)

    if np.isnan(losses):
        raise ValueError('Failed to train model.')

    ray_tune.report(objective=losses)


def _train(
    name: str,
    X_train_C: np.ndarray,
    X_train_N: np.ndarray,
    y_train: np.ndarray,
    X_test_C: np.ndarray,
    X_test_N: np.ndarray,
    y_test: np.ndarray,
    C_cat: np.ndarray,
    C_num: np.ndarray,
    fit: Callable[[pd.DataFrame, np.ndarray, Dict[str, any], List[str]], BaseEstimator],
    encoder: Union[OneHotEncoder, OrdinalEncoder] = None,
    normalize: bool = False,
    select: bool = False,
    select_threshold: float = 1e-5,
    select_params: Dict[str, any] = None,
    oversample: bool = False,
    tune: bool = False,
    tune_search_space: Callable[[optuna.Trial], None] = None,
    tune_split: str = None,
    tune_split_params=None,
    tune_trials: int = -1,
    tune_time_s: int = 60 * 5,
    tune_path: str = './',
    random_state: int = None,
    ignore_warning: bool = False,
    verbosity: int = 0
) -> FoldResult:
    def _log(_v, _m):
        if _v >= verbosity:
            log(_m)
    
    with warnings.catch_warnings():
        if ignore_warning:
            warnings.simplefilter('ignore')
        _log(1, f'Training is started in the fold, {name}')
        
        ELAPSED_TIME = time.time()
        X_train_C, X_train_N = X_train_C.astype(object), X_train_N.astype(np.float32)
        X_test_C, X_test_N = X_test_C.astype(object), X_test_N.astype(np.float32)

        if select:
            _log(0, f'-- Begin feature selection.')
            try:
                select_params = select_params or dict()

                scaler = StandardScaler().fit(X_train_N)
                svc = LinearSVC(
                    random_state=random_state, **select_params
                ).fit(X=np.nan_to_num(scaler.transform(X_train_N)), y=y_train)
                M = np.asarray([abs(r) > select_threshold for r in np.ravel(svc.coef_)])
                C_num = C_num[M]
                X_train_N, X_test_N = X_train_N[:, M], X_test_N[:, M] 
                _log(0, f'-- Complete feature selection: # Cat. columns = {len(C_cat)}; # Num. columns = {len(C_num)}')
            except:
                _log(2, f'-- Fail to select features. Keep training without feature selection. Caused by: \n{tb.format_exc()}')
        
        if normalize:
            _log(0, f'Begin normalizing numeric features.')

            try:
                scaler = StandardScaler().fit(X_train_N)
                X_train_N = np.nan_to_num(scaler.transform(X_train_N)).astype(np.float32)
                X_test_N = np.nan_to_num(scaler.transform(X_test_N)).astype(np.float32)
                _log(0, f'Complete to normalizing numeric features.')
            except:
                _log(2, f'Fail to normalize numeric features. Keep training without it. Caused by: \n{tb.format_exc()}')

        if oversample:
            _log(0, f'Begin oversampling.')
            try:
                ord_enc = OrdinalEncoder(dtype=np.float32).fit(X_train_C)
                X_train = np.concatenate((ord_enc.transform(X_train_C), X_train_N), axis=1)
                I_cat = np.arange(X_train_C.shape[1])
                I_num = np.setdiff1d(np.arange(X_train.shape[1]), I_cat)
                if len(C_cat):
                    sampler = SMOTENC(categorical_features=I_cat, random_state=random_state)
                else:
                    sampler = SMOTE(random_state=random_state)
                X_train, y_train = sampler.fit_resample(X_train, y_train)
                
                X_train_C, X_train_N = X_train[:, I_cat], X_train[:, I_num]
                X_train_C = ord_enc.inverse_transform(X_train_C)
                _log(0, f'Complete oversampling.')
            except:
                _log(2, f'Fail to oversample. Keep training without oversampling. Caused by: \n{tb.format_exc()}')
        X_train_C, X_test_C = encoder.transform(X_train_C).astype(np.int32), encoder.transform(X_test_C).astype(np.int32)

        if isinstance(encoder, OneHotEncoder):
            categories = C_cat = list(encoder.get_feature_names_out(C_cat))
        else:
            categories = {k: {c: i for i, c in enumerate(v)} for k, v in zip(C_cat, encoder.categories_)}
            
        X_train_C = pd.DataFrame(X_train_C, columns=C_cat, dtype=np.int32)
        X_train_N = pd.DataFrame(X_train_N, columns=C_num, dtype=np.float32)
        X_test_C = pd.DataFrame(X_test_C, columns=C_cat, dtype=np.int32)
        X_test_N = pd.DataFrame(X_test_N, columns=C_num, dtype=np.float32)

        X_train = pd.concat((X_train_C, X_train_N), axis=1)
        X_test = pd.concat((X_test_C, X_test_N), axis=1)

        best_config = dict()

        if tune and tune_search_space and tune_split:
            _log(0, f'Begin hyperparamter tuning.')

            tune_split_params = tune_split_params or dict()
            tune_splitter = partial(
                _split, alg=tune_split, random_state=random_state,
                **tune_split_params
            )

            try:
                objective = ray_tune.with_parameters(
                    _trainable,
                    X=X_train, 
                    y=y_train,
                    split=tune_splitter,
                    fit=fit,
                    categories=list(C_cat),
                    ignore_warning=ignore_warning,
                )
                search = OptunaSearch(
                    search_space=tune_search_space,
                    metric='objective',
                    mode='min',
                    sampler=TPESampler(seed=random_state)
                )

                analysis = ray_tune.run(
                    objective,
                    search_alg=search,
                    metric='objective',
                    mode='min',
                    local_dir=tune_path,
                    name=name,
                    verbose=1,
                    raise_on_failed_trial=False,
                    stop=dict(objective=1e-5),
                    time_budget_s=tune_time_s,
                    num_samples=tune_trials
                )
                best_config = analysis.best_config or dict()
                _log(0, f'Complete hyperparamter tuning.')

            except:
                _log(2, f'Fail to tune hyperparameters. Keep training with initial parameters. Caused by: \n{tb.format_exc()}')

        try:
            _log(0, f'Begin model fitting.')
            estimator = fit(X_train, y_train, best_config, list(C_cat))
            result = FoldResult(
                name=name,
                estimator=estimator,
                X_train=X_train,
                y_train=y_train,
                X_test=X_test,
                y_test=y_test,
                tuned_params=best_config,
                categories=categories
            )
            return result
        except:
            _log(2, f'Fail to fit a model. Caused by: \n{tb.format_exc()}')
            return None
        finally:
            _log(1, f'Complete training in the fold, {name} ({time.time() - ELAPSED_TIME:.2f} s).')


def cross_val(
    X: pd.DataFrame,
    y: np.ndarray,
    groups: np.ndarray,
    path: str,
    name: str,
    fit: Callable[[pd.DataFrame, np.ndarray, Dict[str, any], List[str]], BaseEstimator],
    categories: List[str] = None,
    normalize: bool = False,
    split: str = None,
    split_params: Dict[str, any] = None,
    select: bool = False,
    select_threshold: float = 1e-5,
    select_params: Dict[str, any] = None,
    oversample: bool = False,
    tune: bool = False,
    tune_search_space: Callable[[optuna.Trial], None] = None,
    tune_split: str = None,
    tune_split_params=None,
    tune_trials: int = -1,
    tune_time_s: int = 60 * 5,
    tune_path: str = './',
    random_state: int = None,
    ignore_warning: bool = False,
    onehot: bool = False,
    verbosity: int = 0
):
    if not os.path.exists(path):
        raise ValueError('"path" does not exist.')
    
    if not split:
        raise ValueError('"split" should be specified.')
    
    if not ray.is_initialized():
        raise EnvironmentError('"ray" should be initialized.')
    
    jobs = []
    func = ray.remote(_train).remote

    categories = list() if categories is None else categories
    C_cat = np.asarray(sorted(categories))
    C_num = np.asarray(sorted(X.columns[~X.columns.isin(C_cat)]))

    # Encoding categorical features
    if onehot:
        encoder = OneHotEncoder(dtype=np.int32, sparse=False, drop=None).fit(X[C_cat].values)
    else:
        encoder = OrdinalEncoder(dtype=np.int32).fit(X[C_cat].values)

    split_params = split_params or dict()
    splitter = _split(alg=split, X=X, y=y, groups=groups, random_state=random_state, **split_params)

    for idx_fold, (I_train, I_test) in enumerate(splitter):
        if split == 'logo':
            FOLD_NAME = str(np.unique(groups[I_test]).item(0))
        else:
            FOLD_NAME = str(idx_fold + 1)

        X_train, y_train = X.iloc[I_train, :], y[I_train]
        X_train_C, X_train_N = X_train[C_cat], X_train[C_num]

        X_test, y_test = X.iloc[I_test, :], y[I_test]
        X_test_C, X_test_N = X_test[C_cat], X_test[C_num]

        job = func(
            name=f'{name}.{FOLD_NAME}',
            X_train_C=X_train_C.values,
            X_train_N=X_train_N.values,
            y_train=y_train,
            X_test_C=X_test_C.values,
            X_test_N=X_test_N.values,
            y_test=y_test,
            C_cat=C_cat,
            C_num=C_num,
            fit=fit,
            encoder=encoder,
            normalize=normalize,
            select=select,
            select_threshold=select_threshold,
            select_params=select_params,
            oversample=oversample,
            tune=tune,
            tune_search_space=tune_search_space,
            tune_split=tune_split,
            tune_split_params=tune_split_params,
            tune_trials=tune_trials,
            tune_time_s=tune_time_s,
            tune_path=tune_path,
            random_state=random_state,
            ignore_warning=ignore_warning,
            verbosity=verbosity
        )
        jobs.append(job)

    jobs = ray.get(jobs)

    dump(jobs, os.path.join(path, f'{name}.pkl'))


### Measurements for model evaluation

* For the origianl class distribution
    * Majority Voting from DummyClassifier
* For nominal class measures:
    * Accuracy, Balanced Accuracy, F1 (Y), F1 (N)
* For ranking measures:
    * AU-ROC (ref. = 0.5)

In [ ]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, \
    confusion_matrix, precision_recall_fscore_support, \
    roc_auc_score, matthews_corrcoef, average_precision_score, \
    log_loss, brier_score_loss
import numpy as np
from itertools import product
import scipy.stats.mstats as ms
from typing import Dict


def evaluate(
    y_true: np.ndarray,
    y_pred: np.ndarray,
    y_prob: np.ndarray,
    classes: np.ndarray
) -> Dict[str, any]:
    R = {}
    n_classes = len(classes)
    is_multiclass = n_classes > 2

    R['inst'] = len(y_true)
    
    for c in classes:
        R[f'inst_{c}'] = np.sum(y_true == c)

    C = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=classes)
    for (i1, c1), (i2, c2) in product(enumerate(classes), enumerate(classes)):
        R[f'true_{c1}_pred_{c2}'] = C[i1, i2]

    # Threshold Measure
    R['acc'] = accuracy_score(y_true=y_true, y_pred=y_pred)
    R['bac'] = balanced_accuracy_score(y_true=y_true, y_pred=y_pred)
    R['gmean'] = ms.gmean(np.diag(C) / np.sum(C, axis=1))
    R['mcc'] = matthews_corrcoef(y_true=y_true, y_pred=y_pred)
    if is_multiclass:
        for avg in ('macro', 'micro'):
            pre, rec, f1, _ = precision_recall_fscore_support(
                y_true=y_true,
                y_pred=y_pred,
                labels=classes,
                average=avg, 
                zero_division=0
            )
            R[f'pre_{avg}'] = pre
            R[f'rec_{avg}'] = rec
            R[f'f1_{avg}'] = f1
    else:
        for c in classes:
            pre, rec, f1, _ = precision_recall_fscore_support(
                y_true=y_true, y_pred=y_pred, pos_label=c, average='binary', zero_division=0
            )
            R[f'pre_{c}'] = pre
            R[f'rec_{c}'] = rec
            R[f'f1_{c}'] = f1

    # Ranking Measure
    if is_multiclass:
        for avg, mc in product(('macro', 'micro'), ('ovr', 'ovo')):
            R[f'roauc_{avg}_{mc}'] = roc_auc_score(
                y_true=y_true, y_score=y_prob,
                average=avg, multi_class=mc, labels=classes
            )
    else:
        R[f'roauc'] = roc_auc_score(
            y_true=y_true, y_score=y_prob[:, 1], average=None
        )
        for i, c in enumerate(classes):
            R[f'prauc_{c}'] = average_precision_score(
                y_true=y_true, y_score=y_prob[:, i], pos_label=c, average=None
            )
            R[f'prauc_ref_{c}'] = np.sum(y_true == c) / len(y_true)

    # Probability Measure
    R['log_loss'] = log_loss(y_true=y_true, y_pred=y_prob, labels=classes, normalize=True)

    if not is_multiclass:
        R[f'brier_loss'] = brier_score_loss(
            y_true=y_true, y_prob=y_prob[:, 1], pos_label=classes[1]
        )

    return R

### Building and evaluating the models

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from functools import partial
from itertools import combinations, chain
import altair as alt


RANDOM_STATE = 42

FIT_DUMMY = lambda X, y, params, _: DummyClassifier(strategy='prior').fit(X, y)

FIT_RF = lambda X, y, params, _: RandomForestClassifier(random_state=RANDOM_STATE, **params).fit(X, y)

FIT_XGB = lambda X, y, params, _: ModifiedXGBClassifier(
    random_state=RANDOM_STATE, 
    eval_set=True, eval_metric='logloss', early_stopping_rounds=10,
    objective='binary:logistic', verbosity=0,
    **params
).fit(X, y)

FITS = [
    ('DUMMY', FIT_DUMMY, True, False),
    ('RF', FIT_RF, True, True),
    ('XGB', FIT_XGB, True, True)
]

CROSS_VAL_BASE = partial(
    cross_val,
    normalize=True,
    select=True, 
    select_threshold=1e-3,
    select_params=dict(tol=1e-3, max_iter=5000, dual=False, penalty='l1', loss='squared_hinge', C=1e-2),
    tune=False,
    oversample=True,
    random_state=RANDOM_STATE,
    ignore_warning=True,
    verbosity=1
)

LABELS = ['val_fix', 'aro_fix', 'sts_fix', 'dst_fix']

#### 10-Fold CV

In [ ]:
from functools import reduce
from itertools import product
import numpy as np

with on_ray(ignore_reinit_error=True, num_cpus=7):
    for l, (model, fit, onehot, oversample) in product(LABELS, FITS):
        DATA = load(f'./proc/dataset.{l}.pkl')
        X, y, groups = DATA['X'], DATA['y'], DATA['group']
        C = X.columns
        M = np.ones(len(C), dtype=np.bool8)

        CROSS_VAL_BASE(
            name= f'{l}.{model}',
            path='./eval/kfold', 
            X=X, 
            y=y, 
            groups=groups,
            fit=fit, 
            split='kfold',
            split_params=dict(
                n_splits=10
            ),
            categories=list(C[X.dtypes == object]),
            onehot=onehot,
            oversample=oversample
        )

#### Group 10-Fold CV

In [ ]:
from functools import reduce
from itertools import product
import numpy as np


with on_ray(ignore_reinit_error=True, num_cpus=7):
    for l, (model, fit, onehot, oversample) in product(LABELS, FITS):
        DATA = load(f'./proc/dataset.{l}.pkl')
        X, y, groups = DATA['X'], DATA['y'], DATA['group']
        C = X.columns
        M = np.ones(len(C), dtype=np.bool8)

        CROSS_VAL_BASE(
            name= f'{l}.{model}',
            path='./eval/groupk', 
            X=X, 
            y=y, 
            groups=groups,
            fit=fit, 
            split='groupk',
            split_params=dict(
                n_splits=10
            ),
            categories=list(C[X.dtypes == object]),
            onehot=onehot,
            oversample=oversample
        )

## Performance evaluation

In [ ]:
import altair as alt


def vis_measure(X: pd.DataFrame, measure: str):
    X = X.loc[lambda x: (x['metric'] == measure), :]
    
    return alt.Chart(X).mark_boxplot().encode(
        x=alt.X('model:N', title='Model'),
        y=alt.Y('value:Q', title='Value'),
        column=alt.Column('label:N', title='Label')
    )

### 10-Fold CV

In [ ]:
import os
import pandas as pd
import ray
from typing import List


EVAL_KFOLD = []

for p in os.listdir('./eval/kfold/'):
    result = load(os.path.join('./eval/kfold/', p))
    parts = p.split('.')
    label, model = parts[0], parts[1]

    for r in result:
        name = r.name.split('.')[-1]
        X, y = r.X_test, r.y_test
        y_pred = r.estimator.predict(X)
        y_prob = r.estimator.predict_proba(X)
        classes = r.estimator.classes_
        n_features = len(r.X_test.columns)
        measures = evaluate(y, y_pred, y_prob, classes)

        EVAL_KFOLD.append({
            'label': label,
            'model': model,
            'fold': name,
            'n_features': n_features,
            **measures
        })

EVAL_KFOLD = pd.DataFrame(EVAL_KFOLD).melt(
    id_vars=['label', 'model', 'fold'],
    var_name='metric',
    value_name='value'
)

EVAL_KFOLD.to_csv('./eval/kfold-metric.csv', index=False)

In [ ]:
from ipywidgets import fixed, interact
import pandas as pd


EVAL_KFOLD = pd.read_csv('./eval/kfold-metric.csv')
interact(
    vis_measure,
    X=fixed(EVAL_KFOLD),
    measure=EVAL_KFOLD['metric'].unique()
)

### Group 10-Fold CV

In [ ]:
import os
import pandas as pd
import ray
from typing import List


EVAL_GROUPK = []

for p in os.listdir('./eval/groupk/'):
    result = load(os.path.join('./eval/groupk/', p))
    if p.endswith('csv'):
        continue
    parts = p.split('.')
    label, model = parts[0], parts[1]

    for r in result:
        name = r.name.split('.')[-1]
        X, y = r.X_test, r.y_test
        y_pred = r.estimator.predict(X)
        y_prob = r.estimator.predict_proba(X)
        classes = r.estimator.classes_
        n_features = len(r.X_test.columns)
        measures = evaluate(y, y_pred, y_prob, classes)

        EVAL_GROUPK.append({
            'label': label,
            'model': model,
            'fold': name,
            'n_features': n_features,
            **measures
        })

EVAL_GROUPK = pd.DataFrame(EVAL_GROUPK).melt(
    id_vars=['label', 'model', 'fold'],
    var_name='metric',
    value_name='value'
)
EVAL_GROUPK.to_csv('./eval/groupk-metric.csv', index=False)

In [ ]:
from ipywidgets import fixed, interact


EVAL_GROUPK = pd.read_csv('./eval/groupk-metric.csv')
interact(
    vis_measure,
    X=fixed(EVAL_GROUPK),
    measure=EVAL_GROUPK['metric'].unique()
)

## Explanation

### Feature importance

In [ ]:
label_map = {"val": "Valence",
             "aro": "Arousal",
             "sts": "Stress",
             "dst": "Disturbance"
            }

#### 10-Fold CV

In [ ]:
import os
import pandas as pd
import numpy as np
import ray
from typing import List
from sklearn.dummy import DummyClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier


IMP_KFOLD = []

for p in os.listdir('./eval/kfold/'):
    result = load(os.path.join('./eval/kfold/', p))
    parts = p.split('.')
    label, model = parts[0], parts[1]
    
    for r in result:
        estimator = r.estimator
        if isinstance(estimator, DummyClassifier):
            continue
            
        name = r.name.split('.')[-1]
        cols = r.X_test.columns
        
        if isinstance(estimator, RandomForestClassifier):
            imp = estimator.feature_importances_
        elif isinstance(estimator, CalibratedClassifierCV):
            imp = []
            
            for c in estimator.calibrated_classifiers_:
                base_estimator = c.base_estimator
                if hasattr(base_estimator, 'feature_importances_'):
                    imp.append(base_estimator.feature_importances_)
                else:
                    imp.append(base_estimator.model.feature_importances_)
            imp = np.mean(np.vstack(imp), axis=0)
        else:
            imp = estimator.model.feature_importances_
        
        imp = {k: v for k, v in zip(cols, imp)}

        IMP_KFOLD.append({
            'label': label,
            'model': model,
            'fold': name,
            **imp
        })

IMP_KFOLD = pd.DataFrame(IMP_KFOLD).melt(
    id_vars=['label', 'model', 'fold'],
    var_name='feature',
    value_name='importance'
).fillna(0)

IMP_KFOLD.to_csv('./eval/kfold-importance.csv', index=False)

In [ ]:
import pandas as pd
import altair as alt
from itertools import product


IMP_KFOLD = pd.read_csv('./eval/kfold-importance.csv')

TOP_IMP_KFOLD = IMP_KFOLD.groupby(
    ['label', 'model', 'feature'], as_index=False
).mean().sort_values(
    ['label', 'model', 'importance'], ascending=False
).groupby(
    ['label', 'model'], as_index=False
).head(10)

charts = []

for label in TOP_IMP_KFOLD['label'].unique():
    rows = []
    
    for model in TOP_IMP_KFOLD['model'].unique():
        X = TOP_IMP_KFOLD.loc[
            lambda x: (x['label'] == label) & (x['model'] == model), :
        ]
        c = alt.Chart(X).mark_bar().encode(
            x=alt.X('feature:N', title='', sort='-y'),
            y=alt.Y('importance:Q', title='Importance')
        ).properties(
            title=f'{label_map[label.split("_")[0]]}-{model}',
            width=200,
            height=100
        )
        rows.append(c)
    rows = alt.hconcat(*rows)
    charts.append(rows)

alt.vconcat(*charts)

#### Group 10-Fold CV

In [ ]:
import os
import pandas as pd
import numpy as np
import ray
from typing import List
from sklearn.dummy import DummyClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier


IMP_GROUPK = []

for p in os.listdir('./eval/groupk/'):
    result = load(os.path.join('./eval/groupk/', p))
    parts = p.split('.')
    label, model = parts[0], parts[1]
    
    for r in result:
        estimator = r.estimator
        if isinstance(estimator, DummyClassifier):
            continue
            
        name = r.name.split('.')[-1]
        cols = r.X_test.columns
        
        if isinstance(estimator, RandomForestClassifier):
            imp = estimator.feature_importances_
        elif isinstance(estimator, CalibratedClassifierCV):
            imp = []
            
            for c in estimator.calibrated_classifiers_:
                base_estimator = c.base_estimator
                if hasattr(base_estimator, 'feature_importances_'):
                    imp.append(base_estimator.feature_importances_)
                else:
                    imp.append(base_estimator.model.feature_importances_)
            imp = np.mean(np.vstack(imp), axis=0)
        else:
            imp = estimator.model.feature_importances_
        
        imp = {k: v for k, v in zip(cols, imp)}

        IMP_GROUPK.append({
            'label': label,
            'model': model,
            'fold': name,
            **imp
        })

IMP_GROUPK = pd.DataFrame(IMP_GROUPK).melt(
    id_vars=['label', 'model', 'fold'],
    var_name='feature',
    value_name='importance'
).fillna(0)

IMP_GROUPK.to_csv('./eval/groupk-importance.csv', index=False)

In [ ]:
import pandas as pd
import altair as alt
from itertools import product


IMP_GROUPK = pd.read_csv('./eval/groupk-importance.csv')

TOP_IMP_GROUPK = IMP_GROUPK.groupby(
    ['label', 'model', 'feature'], as_index=False
).mean().sort_values(
    ['label', 'model', 'importance'], ascending=False
).groupby(
    ['label', 'model'], as_index=False
).head(10)

charts = []

for label in TOP_IMP_GROUPK['label'].unique():
    rows = []
    
    for model in TOP_IMP_GROUPK['model'].unique():
        X = TOP_IMP_GROUPK.loc[
            lambda x: (x['label'] == label) & (x['model'] == model), :
        ]
        c = alt.Chart(X).mark_bar().encode(
            x=alt.X('feature:N', title='', sort='-y'),
            y=alt.Y('importance:Q', title='Importance')
        ).properties(
            title=f'{label_map[label.split("_")[0]]}-{model}',
            width=200,
            height=100
        )
        rows.append(c)
    rows = alt.hconcat(*rows)
    charts.append(rows)

alt.vconcat(*charts)